# HUMANNOTATOR
*Example notebook*  
  
Build easy custom annotators for your Jupyter/pandas workflow!

In [1]:
import sys
sys.path.insert(0, '../')
from humannotator import Annotator, task_factory, load_data
import pandas as pd

---
### Load the data
You can pass a `list`, `dict`, `Series` or `DataFrame` object into the Annotator.  
Here we will load a dataframe.

In [2]:
df = pd.read_csv('news.csv', index_col=0)
data = load_data(df, item_cols=['title', 'date'], id_col='news_id')

### Set up some tasks
We can create tasks using the `task_factory`.  
Finally, we instantiate the annotator by passing it our data and tasks.

In [3]:
choices={
    '0': 'not adverse media',
    '1': 'adverse media',
    '3': 'exclude from dataset',
}
instruct = "Is the topic political?"
task1 = task_factory(choices, 'Adverse media')
task2 = task_factory('bool', 'Political', instruction=instruct, nullable=True) 

annotator = Annotator(data, [task1, task2])

---
### Run the annotator by calling it
The annotator keeps track of where you were.  
Pass the annotator a list of ids if you only want to annotate specific records.  
You can exit the annotator and it will continue where you left of when you run it again.

In [10]:
annotator()

---
### Access your annotations
The annotations are stored in a dataframe.

In [5]:
annotator.annotated

,Adverse media,Political,timestamp
052632_2015-02-28,adverse media,True,2019-09-02 08:59:15.926451
071607_2016-12-12,exclude from dataset,None,2019-09-02 08:59:25.165664
141694_2016-02-10,not adverse media,False,2019-09-02 08:59:33.171530
137157_2017-02-09,not adverse media,True,2019-09-02 08:59:42.894561
034187_2016-09-27,adverse media,False,2019-09-02 08:59:52.742221


### Merge your annotations with the data

In [6]:
annotator.merged()

,title,date,text,Adverse media,Political,timestamp
052632_2015-02-28,Rand Paul wins 2015 CPAC straw poll,2015-02-28,[Washington (CNN)Sen. Rand Paul won the Conser...,adverse media,True,2019-09-02 08:59:15.926451
071607_2016-12-12,Can Singing Mice Reveal the Roots of Human Spe...,2016-12-12,"[One chilly day in February 1877, a British co...",exclude from dataset,None,2019-09-02 08:59:25.165664
141694_2016-02-10,Dollar hits 15-month low against yen after Yel...,2016-02-10,The dollar fell to a 15-month low against the...,not adverse media,False,2019-09-02 08:59:33.171530
137157_2017-02-09,Trump's Supreme Court pick dispirited by presi...,2017-02-09,"Donald Trump's Supreme Court nominee, Neil Go...",not adverse media,True,2019-09-02 08:59:42.894561
034187_2016-09-27,FULL TEXT: 10 Things Milo Hates About Islam - ...,2016-09-27,"I’m Milo Yiannopoulos, thank you for coming. T...",adverse media,False,2019-09-02 08:59:52.742221


### Save and load your data

In [7]:
annotator.save('annotator.pkl')

In [8]:
annotator2 = Annotator.load('annotator.pkl')

In [9]:
annotator2.annotated

,Adverse media,Political,timestamp
052632_2015-02-28,adverse media,True,2019-09-02 08:59:15.926451
071607_2016-12-12,exclude from dataset,None,2019-09-02 08:59:25.165664
141694_2016-02-10,not adverse media,False,2019-09-02 08:59:33.171530
137157_2017-02-09,not adverse media,True,2019-09-02 08:59:42.894561
034187_2016-09-27,adverse media,False,2019-09-02 08:59:52.742221
